In [1]:
import sits.SITS as sits
import folium
from datetime import datetime
import time

In [2]:
import geopandas as gpd
import pandas as pd

in_vect = "france_barkbeetle_lite_single.geojson"
test_2154 = sits.Vec2gdf(in_vect)
test_2154.gdf.crs.to_epsg()

2154

In [3]:
# calculates the boundiug box for each feature.
test_2154.set_bbox('gdf')
test_2154.bbox.crs.to_epsg()

2154

In [4]:
#f = folium.Figure(height=400)
#m = folium.Map(location=[55, 0], zoom_start=3).add_to(f)
#test_2154.gdf.explore(m=m, height=400, color='green')
#test_2154.bbox.explore(m=m, height=400)

In [5]:
type(test_2154.bbox)

geopandas.geodataframe.GeoDataFrame

In [6]:
test_process = test_2154.bbox.to_crs(4326)

test_process = pd.concat([test_process, test_process['geometry'].bounds], axis=1)
test_process['bbox_4326'] = test_process[['minx', 'miny', 'maxx', 'maxy']].values.tolist()


In [7]:
test_3035 = test_2154.bbox.to_crs(3035)
test_3035_bounds = pd.concat([test_3035, test_3035['geometry'].bounds], axis=1)
test_3035_bounds['bbox_3035'] = test_3035_bounds[['minx', 'miny', 'maxx', 'maxy']].values.tolist()

test_process = pd.concat([test_process, test_3035_bounds['bbox_3035']], axis=1)
test_process['bbox_tuple'] = test_process.apply(lambda row: (row['bbox_4326'], row['bbox_3035']), axis=1)
test_process['bbox_tuple'][0]


([2.8255044256228508,
  45.634941277411386,
  2.8260189959120745,
  45.6353022635073],
 [3761787.7849638816,
  2529437.8009796273,
  3761831.262662839,
  2529481.4105150606])

In [8]:
#%%time

#imgcoll = sits.StacAttack()
#imgcoll.searchItems(total_bbox_4326, date_start=datetime(2023, 1, 1), date_end=datetime(2023, 12, 31))

In [9]:
#len(imgcoll.items)

In [10]:
from dask import delayed, compute

In [11]:
def f_test(aoi_bounds_latlong, aoi_bounds_proj, gid, rep):
    # loads time-series images in EPSG:3035
    imgcoll = sits.StacAttack()
    imgcoll.searchItems(aoi_bounds_latlong, date_start=datetime(2023, 1, 1), date_end=datetime(2023, 12, 31))
    imgcoll.loadImgs(aoi_bounds_proj, crs_out=3035)
    # exports time-series into csv file and netCDF file
    imgcoll.to_nc(f"/eos/jeodpp/home/users/osekenj/data/temp_data/{rep}", gid, 'image')

    

In [12]:
for i in test_process['bbox_tuple'][:20]:
    print (i[1])

[3761787.7849638816, 2529437.8009796273, 3761831.262662839, 2529481.4105150606]
[4019438.9740521642, 2504995.883254013, 4019471.590462089, 2505028.4940533843]
[3990846.743521249, 2827079.0831763, 3990880.024332588, 2827142.598343773]
[3990849.826223296, 2827004.636245847, 3990877.832390555, 2827088.324802132]
[3991289.9031796386, 2827058.74595494, 3991390.4657415943, 2827163.005502219]
[3991028.9235258377, 2826628.4510618323, 3991063.193960321, 2826641.9578250065]
[3967053.295668171, 2853454.003526762, 3967090.996238946, 2853500.4930631756]
[3967023.8647004236, 2853506.1117294356, 3967057.189913272, 2853527.102907303]
[4019577.432066314, 2934944.5341832656, 4019652.0517830993, 2934999.055088769]
[4019839.553133474, 2934765.816373214, 4019921.141183831, 2934858.5466706147]
[4019793.3397727082, 2934708.6873220964, 4019876.4962251536, 2934762.05659873]
[4019592.6021670396, 2935080.3477080576, 4019642.182089403, 2935132.4464516807]
[3983000.8162814267, 2918047.9374025525, 3983022.558951804

In [13]:
%%time
#test 01

for gid, i in enumerate(test_process['bbox_tuple'][:20]):
    f_test(i[0], i[1], gid, "test01")


/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


CPU times: user 17min 30s, sys: 11min 6s, total: 28min 36s
Wall time: 10min 43s


In [14]:
%%time
#test 02
"""
fetch_dask = []
    
for gid, i in enumerate(test_process['bbox_tuple'][:20]):
    single = delayed(f_test)(i[0], i[1], gid, "test02")
    fetch_dask.append(single)

results_dask = compute(*fetch_dask)#, scheduler='processes')
"""

CPU times: user 19 µs, sys: 12 µs, total: 31 µs
Wall time: 56.5 µs


'\nfetch_dask = []\n    \nfor gid, i in enumerate(test_process[\'bbox_tuple\'][:20]):\n    single = delayed(f_test)(i[0], i[1], gid, "test02")\n    fetch_dask.append(single)\n\nresults_dask = compute(*fetch_dask)#, scheduler=\'processes\')\n'

In [15]:
%%time
#test 03

fetch_dask = []
    
for gid, i in enumerate(test_process['bbox_tuple'][:20]):
    single = delayed(f_test)(i[0], i[1], gid, "test03")
    fetch_dask.append(single)

results_dask = compute(*fetch_dask, scheduler='processes')

/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11

CPU times: user 66.6 ms, sys: 22.6 ms, total: 89.2 ms
Wall time: 4min 10s
